<h1>PRÁCTICA 1 - FUNDAMENTOS DE APRENDIZAJE AUTOMÁTICO</h1>
<h3>Realizada la práctica por:<br/>
    <ol>
    -Pablo Díez del Pozo<br/>
    -Alejandro Alcalá Álvarez
    </ol>
 </h3>
<h3>Grupo: 1462</h3>
<h3>Pareja: 01</h3>

In [2]:
#!/usr/bin/python3
# -*- coding: utf-8 -*-

<h3>Importaciones necesarias para la ejecucion del código</h3>
<p>Podemos observar todos los import necesarios que tenemos que realizar para que la ejecución de nuestro codigo funcione a la perfección, a continuación, explicaremos cada uno de los imports y para que son necesarios:</p>
<ol>
    <li>Random: se utiliza para hacer las secuencias de índices aleatorios para las particiones de entrenamiento y de clasificación.
    <li>Math: se utiliza para hacer la distribución normal para los atributos que sean continuos y asi poder calcular su probabilidad.
    <li>Numpy: Es la libreria mas utilizada en esta práctica, debido a que almacenamos los datos en una matriz numpy y guardamos las probabilidades posterioris de los atributos en un array de matrices de numpy.
    <li>ABC: se utiliza para haces clases y métodos abstractos.
    <li>Datos: se utiliza para importar toda la funcionalidad de nuestro modulo Datos.
    <li>Collections: se utiliza para contabilizar las probabilidades condicionadas y para ver cuantas clases hay en el fichero
    <li> SortedDict: se utiliza para ordenar el diccionario que creamos con las probabilidades a priori de cada clase
    <li>Sklearn: se utiliza para hacer el tercer apartado de esta práctica, donde nos da una implementación del algoritmo de Naive-Bayes
    <li>Pyplot: se utiliza en el último apartado de la práctica, donde nos da una implementación para pintar la curva ROC.
     

In [3]:
import random
import math
import numpy as np
from abc import ABCMeta, abstractmethod
from Datos import Datos
from collections import Counter
from sortedcontainers import SortedDict
from sklearn.metrics import confusion_matrix, auc
from matplotlib import pyplot as plt

<h3>Obtener los datos de los Distintos Dataset</h3>
<p>Aqui vamos a poder observar como vamos a codificar los datos que nos dan en un fichero a una matriz Numpy para poder tratar los datos para poder entrenarlos y clasificarlos con Naive-Bayes</p>
<p>Vamos a ver como llamando a la clase Datos y que en su constructor le ponemos la ruta del fichero se crea la matriz numpy de los datos, pero a demás de esa matriz también guardamos información necesaria para poder entrenarlos y clasificarlos correctamente. Por ejemplo, guardamos si los atributos son continuos o discretos.</p>
<p>A continuación, vamos a mostrar una ejecución para cada uno de los conjuntos de datos que nos dan para hacer Naive-Bayes. En la celda de abajo vereis la ejecución.</p>

In [6]:
dataset = Datos('/home/aalcala/Escritorio/FAA_Final3/Practica1/Datasets/lenses.data')
dataset2 = Datos('/home/aalcala/Escritorio/FAA_Final3/Practica1/Datasets/tic-tac-toe.data')
dataset3 = Datos('/home/aalcala/Escritorio/FAA_Final3/Practica1/Datasets/german.data')
print("==============MATRIZ NUMPY DEL CONJUNTO DE DATOS LENSES=====================")
print(dataset.datos)
print("============================================================================")
print("==============MATRIZ NUMPY DEL CONJUNTO DE DATOS TIC-TAC-TOE================")
print(dataset2.datos)
print("============================================================================")
print("==============MATRIZ NUMPY DEL CONJUNTO DE DATOS GERMAN=====================")
print(dataset3.datos)
print("============================================================================")

==============MATRIZ NUMPY DEL CONJUNTO DE DATOS LENSES=====================
[[0. 0. 0. 0. 2.]
 [0. 0. 0. 1. 1.]
 [0. 0. 1. 0. 2.]
 [0. 0. 1. 1. 0.]
 [0. 1. 0. 0. 2.]
 [0. 1. 0. 1. 1.]
 [0. 1. 1. 0. 2.]
 [0. 1. 1. 1. 0.]
 [1. 0. 0. 0. 2.]
 [1. 0. 0. 1. 1.]
 [1. 0. 1. 0. 2.]
 [1. 0. 1. 1. 0.]
 [1. 1. 0. 0. 2.]
 [1. 1. 0. 1. 1.]
 [1. 1. 1. 0. 2.]
 [1. 1. 1. 1. 2.]
 [2. 0. 0. 0. 2.]
 [2. 0. 0. 1. 2.]
 [2. 0. 1. 0. 2.]
 [2. 0. 1. 1. 0.]
 [2. 1. 0. 0. 2.]
 [2. 1. 0. 1. 1.]
 [2. 1. 1. 0. 2.]
 [2. 1. 1. 1. 2.]]
==============MATRIZ NUMPY DEL CONJUNTO DE DATOS TIC-TAC-TOE================
[[2. 2. 2. ... 1. 1. 1.]
 [2. 2. 2. ... 2. 1. 1.]
 [2. 2. 2. ... 1. 2. 1.]
 ...
 [1. 2. 1. ... 1. 2. 0.]
 [1. 2. 1. ... 1. 2. 0.]
 [1. 1. 2. ... 2. 2. 0.]]
==============MATRIZ NUMPY DEL CONJUNTO DE DATOS GERMAN=====================
[[ 0.  6.  4. ...  1.  0.  0.]
 [ 1. 48.  2. ...  0.  0.  1.]
 [ 3. 12.  4. ...  0.  0.  0.]
 ...
 [ 3. 12.  2. ...  0.  0.  0.]
 [ 0. 45.  2. ...  1.  0.  1.]
 [ 1. 45.  4. ...  0

<h3>Apartado1: Estrategia de Particionado</h3>
<p>En este apartado vamos a probar las dos estrategias de particionado de los datos que hemos tenido que implementar en esta práctica, las cuales son:</p>
    <ol>
        <p>- Validación Simple.</p>
        <p>- Validación Cruzada.</p>
    </ol>
<p>Nuestra estrategia de <strong>validación simple</strong> consiste en meterle un porcentaje por el cual queremos dividir el conjunto de datos en dos subconjuntos de datos, donde uno lo vamos a utilizar para entrenar y el otro lo vamos a utilizar para hacer la predicción con nuestro clasificador. En la celda de abajo mostraremos el código necesario para poder realizar correctamente la validacion simple.</p>
<p>Como podemos observar en el código de abajo de validación simple, lo que hacemos es que ponemos una semilla a random y decimos que el numero de particiones va a ser uno. A continuación, haremos un permutacion de numeros aleatorios entre el 0  y el número de datos que hay en el fichero. Por ultimo, lo que hacemos es que le creamos la partición que va a tener en su interior los dos subconjuntos de Train y Test. En esa permutación lo multiplicamos por el porcentaje que le hemos dado nosotros para crear los dos suboconjuntos.</p>
    
Análisis  de  las dosestrategias  de  particionado  propuestas:  simple, y cruzada, para los conjuntos propuestos: lenses, germany tic-tac-toe.El análisis  consiste  en  una  descripción  de  los  índices  de  train  y  test devueltos  por  cada  uno  de  los  métodos  de  particionado,  junto  con  un comentario sobre las ventajas/desventajas de cada uno de ellos.

In [ ]:
class ValidacionSimple(EstrategiaParticionado):

  def __init__(self, porcentaje):
    self.porcentaje = porcentaje
    super().__init__("Validacion simple")

  # Crea particiones segun el metodo tradicional de division de los datos segun el porcentaje deseado.
  # Devuelve una lista de particiones (clase Particion)
  # TODO: implementar
  def creaParticiones(self, datos, seed=None):

    np.random.seed(seed)
    self.numeroParticiones = 1

    # Generamos una lista con todos los números de datos aleatorios
    indicesAleatorios = np.random.permutation(int(datos.numDatos))

    # Creamos la particion, en funcion del porcentaje especificado
    self.particiones = [Particion(indicesAleatorios[:int(datos.numDatos * self.porcentaje)],
                                  indicesAleatorios[int(datos.numDatos * self.porcentaje):])]

    return self.particiones